<a href="https://colab.research.google.com/github/SirTee12/LLM-Kaggle-Google-GenAI/blob/main/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the SDK

In [ ]:
!pip uninstall -qqy jupyterlab # uninstall jupyterlab in the case it conflict google colab base image
!pip install -U -q "google-genai==1.7.0" # install google genai library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.3 MB/s eta 0:00:00


import the SDK and some helpers for rendering the output. The `types` in `import types` may include custom data structures, classes or type hints psecifically to work with google AI. These custom types might represent things like model parameters, input/output formats for AI models or config settings.

In [1]:
from google import genai
from google.genai import types
from google.colab import userdata

import enum
import io
from IPython.display import HTML, Markdown, display, clear_output
import typing_extensions as typing

## API Call Retry Implementation
Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota


1.  **Imports `retry`:** It imports the `retry` module from `google.api_core`, which provides tools for automatic retries of API calls.

2.  **Defines `is_retriable`:**
    * It creates a lambda function called `is_retriable`.
    * This function checks if an exception (`e`) is an instance of `genai.errors.APIErrpr` and if its error code (`e.code`) is either 429 (Too Many Requests) or 503 (Service Unavailable).
    * It returns `True` if both conditions are met (meaning the API call is considered retriable), and `False` otherwise.

3.  **Applies Retry Logic:**
    * It modifies the `genai.models.Models.generate_content` function by wrapping it with the `retry.retry` function.
    * The `retry.retry` function is configured with the `is_retriable` function as the `predicate`. This tells `retry.retry` to only retry the API call if the `is_retriable` function returns `True` for the encountered exception.
    * Essentially, this replaces the original `generate_content` function with a new version that automatically retries on 429 and 503 errors, making the code more robust to temporary API issues.

In [2]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in (429, 503))

genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [3]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#secret_value_0 = user_secrets.get_secret("secret-label-prompt")

## Get the API keys and create a client

Retrieves the API key securely and create a client object that will be used to communicate with the generative AI service, using the retrieved API key for authentication.

In [4]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # retrive the APi key
client = genai.Client(api_key=GOOGLE_API_KEY)   # create a client to interact with the genai application

## Running the First Prompt

In [5]:
# generate a response
response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = "Explain AI to me like I'm a kid"
)

print(response.text)
Markdown(response.text)

Okay, imagine you have a really smart puppy named Sparky.

Sparky doesn't know anything when he's born. You have to teach him everything!

**AI is like teaching a computer to be like Sparky, but instead of teaching him tricks like "sit" or "fetch," you teach him to do things like:**

*   **Recognize things:** You show Sparky lots of pictures of cats and dogs and tell him which is which.  After a while, he learns to tell them apart!  AI can do the same thing, but with anything you show it, like faces, cars, or even different types of fruit!

*   **Learn from mistakes:**  When Sparky chews your shoe, you tell him "no!".  He learns not to do that again.  AI can also learn from its mistakes.  If it guesses the wrong answer, you can tell it, and it will try to do better next time.

*   **Solve problems:**  Maybe you teach Sparky to find a hidden toy.  AI can also solve problems, like figuring out the best way to get to school or writing a story!

**So, what makes AI different from Sparky?**

Okay, imagine you have a really smart puppy named Sparky.

Sparky doesn't know anything when he's born. You have to teach him everything!

**AI is like teaching a computer to be like Sparky, but instead of teaching him tricks like "sit" or "fetch," you teach him to do things like:**

*   **Recognize things:** You show Sparky lots of pictures of cats and dogs and tell him which is which.  After a while, he learns to tell them apart!  AI can do the same thing, but with anything you show it, like faces, cars, or even different types of fruit!

*   **Learn from mistakes:**  When Sparky chews your shoe, you tell him "no!".  He learns not to do that again.  AI can also learn from its mistakes.  If it guesses the wrong answer, you can tell it, and it will try to do better next time.

*   **Solve problems:**  Maybe you teach Sparky to find a hidden toy.  AI can also solve problems, like figuring out the best way to get to school or writing a story!

**So, what makes AI different from Sparky?**

*   **Super Speed:** AI can learn much, much faster than a puppy.
*   **Huge Memory:** AI can remember way more information than Sparky.
*   **No Treats Needed:**  You don't have to give AI treats to keep it learning!

**Where do we see AI?**

*   **Your phone:**  When you ask Siri or Google Assistant a question, that's AI!
*   **Video games:**  The enemies in your games use AI to make decisions.
*   **Movies:** Sometimes special effects artists use AI to make things look really real.

**AI is like a super-smart, fast-learning helper that can do lots of amazing things! It's still learning and growing, but it's already making our lives easier and more fun!**

Does that make sense?


## Let's start a chat

In [6]:
chat = client.chats.create(model = "gemini-2.0-flash", history = [])
response = chat.send_message('Hello, my name is Ahmad')
print(response.text)

Hello Ahmad! It's nice to meet you. How can I help you today?



In [7]:
response = chat.send_message("can you tell me something interesting about dinosaurs")
print(response.text)

Okay, here's an interesting fact about dinosaurs:

**Some dinosaurs had feathers, even the massive ones!**

It's commonly thought that only smaller, bird-like dinosaurs had feathers, but evidence is growing that many different types of dinosaurs, including some very large ones like certain types of ornithopods, may have had feathers or feathery proto-structures. These feathers likely weren't for flight (in the larger species), but rather for insulation, display, or even for tactile purposes.

So, imagine a huge, plant-eating dinosaur, bigger than an elephant, covered in fluffy feathers! It's a pretty mind-blowing thought.

I can tell you more if you'd like! Just let me know what aspects of dinosaurs interest you.



In [8]:
response = chat.send_message("do you still remember my name")
print(response.text)

Yes, I do! Your name is Ahmad.



# Chose a model from the Gemini model family

Retrieves a list of available AI models, searches for a specific model named "gemini-2.0-flash," and if found, displays its details in a formatted JSON-like structure. The search stops once the target model is located.

In [9]:
for model in client.models.list():
  print(model)

name='models/chat-bison-001' display_name='PaLM 2 Chat (Legacy)' description='A legacy text-only model optimized for chat conversations' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=4096 output_token_limit=1024 supported_actions=['generateMessage', 'countMessageTokens']
name='models/text-bison-001' display_name='PaLM 2 (Legacy)' description='A legacy model that understands text and generates text as an output' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=8196 output_token_limit=1024 supported_actions=['generateText', 'countTextTokens', 'createTunedTextModel']
name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, up

In [10]:
from pprint import pprint

for model in client.models.list():
  if model.name == "models/gemini-2.0-flash":
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens', 'createCachedContent'],
 'tuned_model_info': {},
 'version': '2.0'}


## Parameters Exploration

### Output Length

In [11]:
# set the max number of output token to 200
short_config = types.GenerateContentConfig(max_output_tokens = 200)

# create a response
response_short = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = short_config,
    contents='write a short essay on the importance of education in modern day society.'
)

print(response_short.text)

## The Cornerstone of Progress: Why Education Remains Vital in Modern Society

In the labyrinthine complexity of modern society, amidst rapid technological advancements and ever-shifting socio-economic landscapes, one constant remains a beacon of progress and empowerment: education. While access to information has become democratized through the internet, the crucial role of education in cultivating critical thinking, fostering innovation, and ensuring equitable opportunities has only become more pronounced. It is not merely about memorizing facts; rather, education serves as the cornerstone upon which individual fulfillment and societal advancement are built.

Firstly, education empowers individuals with the critical thinking skills necessary to navigate the deluge of information that characterizes the modern world. In an era rife with misinformation and biased narratives, the ability to analyze, evaluate, and discern truth from falsehood is paramount. Education provides the framework

### Temperature

How much randomness is included when selecting the next word (token) in language models depends on the "temperature" parameter.  A higher temperature causes the model to take into account a greater number of potential words, producing more inventive and diverse results.  On the other hand, a lower temperature forces the model to adhere to the most likely terms, producing language that is more concentrated and predictable.  The model turns completely deterministic when the temperature is set to 0, always choosing the word that is the most likely.  Temperature does not, however, ensure actual randomness; rather, it is a means of directing the model toward more or less random results.

In [12]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 2.0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Purple
 _________________________
Turquoise
 _________________________
Magenta
 _________________________
Purple.
 _________________________
Azure
 _________________________


The previous output shows the randomness but we would experiment with temperature value of 0. we can see from the output that it is more direct and precise and there is no randomness.

In [13]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________


### Top P

In [14]:
model_config = types.GenerateContentConfig(
    temperature = 1.0, # default config
    top_p = 0.95       # default config
)

story_prompt = "You are a senior network engineer. Explain ospf to a 5 year old"
response_topp = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents=story_prompt
)

print(response_topp.text)

Okay, imagine we're building a big network of roads for toy cars. We want our toy cars to be able to drive to any house in the neighborhood quickly and easily.

OSPF is like a helpful map and a set of rules we give each intersection so the toy cars know the best way to go!

Here's how it works:

1. **Each Intersection Knows Its Neighbors:**  Imagine each intersection has a little name tag and knows the names of the intersections right next to it. OSPF makes each router (our intersection) know who its direct neighbors (other routers) are.  They say "Hi, I'm Intersection A!" and the other intersections say "Hi, I'm Intersection B!"

2. **Sharing Information:** Then, each intersection tells all the *other* intersections (not just the ones next door) about its neighbors and how long it takes (or how "bumpy" the road is) to get to those neighbors.  So, Intersection A might say, "Okay everyone, I'm connected to Intersection B and it's a smooth, short drive. I'm also connected to Intersection

## Prompting

### Zero-shot Prompt

In [15]:
# set the prompt parameters
model_config = types.GenerateContentConfig(
  temperature = 0.1,
  top_p = 1,
  max_output_tokens = 5
)

# set the prompt
zero_short_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response_zero_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents = zero_short_prompt
)

print(response_zero_shot.text) # print the output as text

POSITIVE



#### Enum Mode
The Sentiment enum acts as a schema or a constraint. It tells the language model that the expected output should be one of these three predefined sentiment categories.

The `text/x.enum` MIME type indicates that the response should be one of the values defined in the response_schema.

In [16]:
# create a new enumeration class tha inherit from enum.Enum
class Sentiment(enum.Enum):
  POSITIVE = 'positive'
  NEUTRAL = 'neutral'
  NEGATIVE = 'negative'

response_zero_short_enum = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(

        # tell the model the ezpected output format and that the rsponse should
        # be one of thos defined in the schema
        response_mime_type = 'text/x.enum',

        # set the Sentiment as schema
        response_schema = Sentiment
    ),

    contents = zero_short_prompt
)
print(response_zero_shot.text)

POSITIVE



### Few Short Prompt

In [17]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = 'Give me a large with cheese and pineapple'

response_one_few_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        max_output_tokens = 250
    ),
    contents = [few_shot_prompt, customer_order]
)

print(response_one_few_shot.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```



#### JSON Mode

`class PizzaOrder(typing.TypedDict):`: This defines a new class called PizzaOrder that inherits from typing.TypedDict. TypedDict is used to create dictionary types where the keys and their corresponding value types are known

Purpose of the `TypedDict` is to act as a schema. It defines the structure and data types of the JSON output we expect from the language model. This allows us to work with the model's response in a structured and predictable way.


In [18]:
class PizzaOrder(typing.TypedDict):
  size:str
  ingredients: list[str]
  type:str

response_json = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 0.1,
        response_mime_type = 'application/json',
        response_schema = PizzaOrder
    ),

    contents = "Can I have a large dessert pizza with apple and chocolate"
)

print(response_json.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


## Chain of Thought

Chain of Thought prompting improves the language model's ability to perform complex reasoning tasks. By explicitly requesting the model to show its reasoning process, it's more likely to arrive at the correct answer, as it avoids impulsive or incorrect conclusions.

In [19]:
# set up the prompt
cot_prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? let's think step by step."""

response_cot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = cot_prompt)

Markdown(response_cot.text)


Here's how to solve the problem step-by-step:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.
2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.
3. **Determine your partner's current age:** Since your partner is 8 years older than you, and you are now 20, your partner is currently 20 + 8 = 28 years old.

**Therefore, your partner is currently 28 years old.**


## Reason and Act: ReAct

In [20]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

In [21]:
# set the question
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences = ['\nObservation'],
    system_instruction = model_instructions + example1 + example2
    )

# Create a chat that has the model instructions and examples pre-seeded.
react_response = client.chats.create(
    model = 'gemini-2.0-flash',
    config = react_config
)

resp = react_response.send_message(question)

print(resp.text)

Thought 1
I need to find the Transformers NLP paper, then find the authors listed, and then find the youngest author.

Action 1
<search>Transformers NLP paper</search>



# Thinking Mode


In [22]:
# set a response
response_thinking = client.models.generate_content_stream(
    model = 'gemini-2.0-flash-thinking-exp',
    contents = 'Who was the youngest athor listed on the transofrmer NLP paper'
)

buffer = io.StringIO()
for chunk in response_thinking:
  buffer.write(chunk.text)

  # display rsponse as it is streamed
  print(chunk.text, end = '')

# render the finished response as formatted markdown
clear_output()
Markdown(buffer.getvalue())

The youngest author listed on the Transformer NLP paper, "Attention is All You Need," is **Aidan N. Gomez**.

While precise birthdates aren't typically public information for researchers, we can infer relative ages based on their career stage at the time of the paper's publication (2017).

* **Aidan N. Gomez** was a PhD student at the University of Toronto and a visiting researcher at Google Brain when the paper was published. Being a PhD student at the time strongly suggests he was likely the youngest author among the group, which primarily consisted of established researchers at Google Brain.

The other authors were:

* Ashish Vaswani
* Noam Shazeer
* Niki Parmar
* Jakob Uszkoreit
* Llion Jones
* Łukasz Kaiser
* Illia Polosukhin

These other authors were all affiliated with Google Brain and likely in more senior research positions at the time, making it highly probable that Aidan N. Gomez, as a PhD student, was the youngest.

## Code Prompting

In [23]:
# initiate a code prompt
code_prompt = """
write a python function to calculate the factorial of a number.
just explain how to write dont provide the code. remember dont write the code,
just give a structure
"""

response_code = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature=1,
        top_p = 1,
        max_output_tokens=1024
    ),
    contents = code_prompt
)


Markdown(response_code.text)

Okay, here's the structure for a Python function to calculate the factorial of a number, without providing the actual code:

**1. Function Definition:**

*   Start by defining a function using the `def` keyword.  Give the function a descriptive name like `factorial`.
*   The function should accept one argument: the number for which you want to calculate the factorial.  Let's call this argument `n`.

**2. Input Validation:**

*   The first step inside the function should be to check if the input `n` is valid. Factorial is typically defined for non-negative integers.  You'll want to handle cases where `n` is:
    *   Negative: Return an appropriate message or raise an exception to signal an error, since factorial is not defined for negative numbers.
    *   Not an integer:  You can either truncate it (only if you decide that is an acceptable behavior), raise an error or decide what is appropriate for the usage.

**3. Base Case:**

*   The factorial of 0 is 1.  This is your base case for the recursion (if you choose to use recursion) or a simple condition in an iterative approach.  If `n` is 0, return 1.

**4. Recursive or Iterative Calculation:**

*   **Recursive Approach:**
    *   If using recursion, the function will call itself.
    *   The factorial of `n` is `n` multiplied by the factorial of `n-1`. So, the function should return `n * factorial(n-1)`.
*   **Iterative Approach:**
    *   If you prefer an iterative approach, you'll use a loop.
    *   Initialize a variable (e.g., `result`) to 1.
    *   Loop from 1 to `n` (inclusive).  In each iteration, multiply `result` by the current loop number.

**5. Return Value:**

*   Finally, the function should return the calculated factorial, stored in the `result` variable (if you used the iterative approach) or obtained from the recursive calls.

**In Summary**

The function needs to:

1.  Define the function with an input argument for the number.
2.  Validate the input to ensure it's a non-negative integer (or handle other cases per your requirements).
3.  Handle the base case (factorial of 0 is 1).
4.  Calculate the factorial either recursively or iteratively.
5.  Return the calculated factorial.


### Code Execution

In [24]:
# set up a config type
config = types.GenerateContentConfig(
    tools = [types.Tool(codeExecution=types.ToolCodeExecution())]
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum
"""

response_exec_code = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = config,
    contents = code_exec_prompt
)

for part in response_exec_code.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print('------------')

{'text': 'Okay, I can do that. First, I need to generate the first 14 odd '
         'prime numbers. Remember that a prime number is a whole number '
         'greater than 1 that has only two divisors: 1 and itself. Also, the '
         'number 2 is the only even prime number. Since you asked for odd '
         'prime numbers, I can skip 2.\n'
         '\n'
         "Here's how I will proceed:\n"
         '\n'
         '1.  Start with the first odd prime number, which is 3.\n'
         '2.  Iterate through odd numbers, checking if they are prime.\n'
         '3.  Add the prime numbers to a list until I have 14 of them.\n'
         '4.  Calculate the sum of the prime numbers in the list.\n'
         '\n'}
------------
{'executable_code': {'code': 'def is_prime(n):\n'
                             '  """Check if a number is prime."""\n'
                             '  if n <= 1:\n'
                             '    return False\n'
                             '  for i in range(2, int(n**

## Explaining code

In [27]:
# specifiy a path to a file content
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh


explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response_explain_code = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = explain_prompt
)

Markdown(response_explain_code.text)

This file is a Bash script designed to enhance your command-line prompt with information about the current Git repository.  It's commonly referred to as a "Git prompt script."

**What it does (High-Level):**

*   **Displays Git Status:**  Shows you the current branch, whether there are uncommitted changes (staged, unstaged, untracked), if you're ahead or behind the remote, and other useful Git-related information directly in your terminal prompt.
*   **Customizable:**  It's highly configurable, allowing you to customize the colors, symbols, and the specific information displayed in the prompt.  You can define your own themes and color schemes.
*   **Asynchronous Updates (Optional):** Can be configured to fetch remote Git status in the background so the prompt doesn't slow down your workflow.

**Why you would use it:**

*   **At-a-glance Git Information:** Provides immediate feedback about the state of your Git repository without needing to run `git status` constantly. This saves time and reduces the risk of committing changes in the wrong state.
*   **Improved Workflow:**  Makes it easier to track your Git activity and avoid common mistakes.
*   **Personalized Prompt:** Allows you to create a prompt that is visually appealing and contains the specific Git information most relevant to you.
*   **Virtual environment support:** Support to show in the prompt when you are inside a python or node environment.

In essence, it makes your terminal prompt "Git-aware" so you're always informed about your repository's status while working in the command line.
